In [1]:
# Import libraries 
import pandas as pd
import numpy as np
import os

In [2]:
# Create path to project folder
path = r'C:\Users\tatal\06-2023 Instacart Basket Analysis'

In [5]:
#IMPORT ords_prods_merged from last 4-8 changes and call ords_prods_merge
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined_busy.pkl'))

In [6]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days,average orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,average orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,most orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,average orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,most orders


In [7]:
ords_prods_merge.shape

(32404859, 18)

## Task 2 - group the entire dabase by depart id.

In [9]:
## Create grop by department_id and agreegated by order number
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [10]:
## perfom mult aggregation
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
1                15.457838   1  99
2                17.277920   1  99
3                17.170395   1  99
4                17.811403   1  99
5                15.215751   1  99
6                16.439806   1  99
7                17.225802   1  99
8                15.340650   1  99
9                15.895474   1  99
10               20.197148   1  99
11               16.170638   1  99
12               15.887671   1  99
13               16.583536   1  99
14               16.773669   1  99
15               16.165037   1  99
16               17.665606   1  99
17               15.694469   1  99
18               19.310397   1  99
19               17.177343   1  99
20               16.473447   1  99
21               22.902379   1  99

##We can see there are orders for all departments, using only the subset we could only see 8 departs. For the subset, the depart with most orders was 16, but now we can see it's dept 21.

## Create a flag for loyalty customers

In [11]:
## create a column max order and transfor the order numb for max count
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [12]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days,average orders,10
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,average orders,10
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,most orders,10
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,average orders,10
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,most orders,10


##Flag the groups for loyalty customers

In [13]:
## loyal more or equal 40 orders max
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [14]:
## regular more than 10 and lower than 40
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [15]:
## New customer less or equal to 10
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [16]:
## Count flag
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

## Statistic for customer flag

In [17]:
# check price min, max and average
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

##New customer usualy pays more for the items. And loyal customer pays less

## Create flags for spending per customer

In [19]:
## create a column mean and transfor the price for mean value
ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [20]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,department_id,prices,_merge,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days,average orders,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,average orders,10,New customer,6.367797
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,most orders,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,Least busy,Least busy days,average orders,10,New customer,6.367797
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,Least busy,Least busy days,most orders,10,New customer,6.367797


In [21]:
ords_prods_merge.shape

(32404859, 21)

## Create spending flag

In [22]:
## If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
ords_prods_merge.loc[ords_prods_merge['mean_price'] < 10, 'spending_flag'] = 'Low spender'

In [23]:
## If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.
ords_prods_merge.loc[ords_prods_merge['mean_price'] >= 10, 'spending_flag'] = 'High spender'

In [24]:
## Count flag spender
ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

In [27]:
# check price min, max and average per spending flag
ords_prods_merge.groupby('spending_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                     mean  min      max
spending_flag                          
High spender   222.343698  1.0  99999.0
Low spender      7.780685  1.0     25.0

## Create an order frequency flag

In [29]:
## create a column median days orders and transfor the days_since_last_order for median value
ords_prods_merge['median_days_last_order'] = ords_prods_merge.groupby(['user_id'])['days_since_last_order'].transform(np.median)

In [30]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days_last_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,both,Mid-range product,Regularly busy,Regularly busy days,average orders,10,New customer,6.367797,Low spender,20.5
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,both,Mid-range product,Regularly busy,Least busy days,average orders,10,New customer,6.367797,Low spender,20.5
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,both,Mid-range product,Regularly busy,Least busy days,most orders,10,New customer,6.367797,Low spender,20.5
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Least busy days,average orders,10,New customer,6.367797,Low spender,20.5
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Least busy days,most orders,10,New customer,6.367797,Low spender,20.5


## Create frequency flag

In [31]:
## If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
ords_prods_merge.loc[ords_prods_merge['median_days_last_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [33]:
## If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.
ords_prods_merge.loc[(ords_prods_merge['median_days_last_order'] <= 20) & (ords_prods_merge['median_days_last_order'] > 10), 'frequency_flag'] = 'Regular customer'

In [34]:
## If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”
ords_prods_merge.loc[ords_prods_merge['median_days_last_order'] < 10, 'frequency_flag'] = 'Frequent customer'

In [35]:
## Count flag spender
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        20287988
Regular customer          7208564
Non-frequent customer     3636437
NaN                       1271870
Name: frequency_flag, dtype: int64

In [36]:
ords_prods_merge.groupby('frequency_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                            mean  min      max
frequency_flag                                
Frequent customer      11.731910  1.0  99999.0
Non-frequent customer  11.943617  1.0  99999.0
Regular customer       12.628375  1.0  99999.0

In [38]:
#export the merge data as pickle with new aggregated columns
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_aggregated_4-8.pkl'))